In [1]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence
import pandas as pd
import numpy as np
np.random.seed(0)
from nltk import word_tokenize
from gensim.models import word2vec

Using TensorFlow backend.


In [2]:
df = pd.read_csv('model_data.csv')

In [3]:
df.drop(columns = ['Unnamed: 0'], inplace= True)

In [4]:
df

,signal_x,retweet_from,tweet_length,encoded_sentiment,polarity,tweet,ability,able,aboard,abort,...,you,your,yours,yourself,yr,yup,zero,zip,zone,zoo
0,stay,137,105,1,0.366667,"Assuming max acceleration of 2 to 3 g's, but i...",0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,stay,64,113,1,0.200000,BFR is capable of transporting satellites to o...,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,stay,137,6,0,0.000000,Yup :),0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0
3,stay,137,7,0,0.000000,Part 2,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,stay,137,96,1,0.650000,Fly to most places on Earth in under 30 mins a...,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
5,stay,64,80,1,0.125000,"Supporting the creation of a permanent, self-s...",0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
6,stay,137,57,-1,-0.166667,BFR will take you anywhere on Earth in less th...,0.0,0.0,0.0,0.0,...,0.271183,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
7,stay,137,84,0,0.000000,Mars City Opposite of Earth. Dawn and dusk sky...,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
8,stay,137,16,-1,-0.800000,Moon Base Alpha,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9,stay,137,60,1,0.357143,Will be announcing something really special at...,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [ ]:
target = df.signal_x

In [ ]:
data = df['tweet'].map(word_tokenize).values

In [ ]:
y = pd.get_dummies(target).values

In [ ]:
tokenizer = text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(list(df.tweet))
list_tokenized_tweets = tokenizer.texts_to_sequences(df.tweet)
X_t = sequence.pad_sequences(list_tokenized_tweets, maxlen=100)

In [ ]:
embedding_size = 128
input_ = Input(shape=(100,))
x = Embedding(10000, embedding_size)(input_)
x = LSTM(25, return_sequences=True)(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.5)(x)
x = Dense(50, activation='relu')(x)
x = Dropout(0.5)(x)
# There are 41 different possible classes, so we use 41 neurons in our output layer
x = Dense(3, activation='softmax')(x)

model = Model(inputs=input_, outputs=x)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 128)          1280000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 25)           15400     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 25)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                1300      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
__________

In [ ]:
model.fit(X_t, y, epochs=2, batch_size=32, validation_split=0.1)

Train on 1981 samples, validate on 221 samples
Epoch 1/2
